# MAC0417/5768 - Visão e Processamento de Imagens (2021)

Exercício Programa 3.2 - Classificação de imagens

Gabriela Melo e Richard Block

Projeto GitHub: https://github.com/gabi-melo/image_processing/tree/main/EP3

Base de imagens: https://www.kaggle.com/gabrielamelo/image-processing


# Resumo

Nessa parte do EP realizamos a etapa de classificação, baseada nas imagens binárias com Feret Box obtidas na primeira parte. As imagens foram redimensionaras para que todas as classes apresentem o mesmo tamanho. Os dados foram separados em conjuntos de treino e de teste. Então foi executada uma etapa de preparação para o aprendizado, com a extração do vetor de características. Para isso, foi utilizada a análise de componentes principais (PCA). A PCA descobre os eixos/vetores de maior variância dos dados, fazendo uma análise a partir da matriz de covariância. O vetor que indica a maior variabilidade dos dados é o primeiro autovetor. O vetor perpendicular é o segundo autovetor, indicando o segundo vetor de maior variabilidade. A transformada PCA muda a coordenada dos pontos definida pelos autovetores, realizando a redução da dimensionalidade.

O vetor de características resultante foi então submetido à classificação, utilizando um algoritmo de máquina de suporte vetorial (SVM). Um classificador define um modelo algébrico para qual é a superfície de decisão. O modelo mais simples é uma linha reta que divide o conjunto de dados para fazer a separação dos dados entre as diferentes classes. As máquinas de suporte vetorial procuram criar uma reta que define uma função critério, a qual é otimizada. O procedimento de otimização faz calcula qual é a linha reta que maximiza as distâncias dos pontos mais próximos da reta. O processo de treinamento de SVM descobre qual é essa linha reta que apresenta a maior distância possível de todas as classes. Esse treino foi realizado exclusivamente nos dados do conjunto de treino.
Foram calculadas métricas de avaliação da qualidade do modelo nos dados do conjunto de teste. Foi realizada a predição dos rótulos das classes do conjunto de teste, e foram computadas variáveis da performance do classificador.

# Download do Kaggle dataset

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [62]:
# download do dataset
%cd /content/gdrive/My Drive/Kaggle
!kaggle datasets download -d gabrielamelo/image-processing/objects

/content/gdrive/My Drive/Kaggle


In [ ]:
# conferir conteúdo do dataset
!ls "/content/gdrive/My Drive/Kaggle/objects"

# Classificação usando SVM

As imagens já foram pré-processadas, todas padronizadas no tamanho (64, 64)

In [72]:
# importar bibliotecas
from glob import glob 
import cv2
from PIL import Image
from skimage import io
from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [81]:
## definir variáveis dos dados

paths = glob('/content/gdrive/My Drive/Kaggle/objects/*.jpg') # path para os objects

# criar variável para os arrays das imagens
X = []

for path in paths:

  an_image = Image.open(path)
  image_sequence = an_image.getdata()
  image_array = np.array(image_sequence)
  X.append(image_array)

X = np.array(X)
print(X)

# criar variável para os rótulos das imagens
y = []

for path in paths:
    
  img_class = path.split('/')[-1].split('_')[0]
  y.append(img_class)

y = np.array(y)
print(y)

[[101  96  89 ... 188 193 171]
 [109  91 148 ... 135 165 170]
 [  9   9   9 ... 106 107 107]
 ...
 [127 111 130 ... 205 203 210]
 [110 109 111 ... 138 139 139]
 [ 44  45  45 ...  26  25  25]]
['livro' 'copo' 'oculos' ... 'chinelo' 'livro' 'escova']


In [82]:
## dividir a amostra em conjuntos de treinamento e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print(X_train.shape, X_test.shape)

(3019, 4096) (1007, 4096)


## Análise de componentes principais (PCA)

In [83]:
# calcular os autovetores/extrair os vetores de características

pca = PCA(n_components=150, whiten=True)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=150, random_state=None,
    svd_solver='auto', tol=0.0, whiten=True)

Os dados de treinamento e de teste projetados nas bases do PCA.

In [84]:
# calcular as dimensões da transformada de PCA

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(X_train_pca.shape)

(3019, 150)


In [85]:
print(X_test_pca.shape)

(1007, 150)


## Classificação

In [87]:
# Treinamento do modelo de classificação SVM

print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf = clf.fit(X_train_pca, y_train) # ajuste da função critério
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
done in 433.592s
Best estimator found by grid search:
SVC(C=1000.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.0001,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


## Métricas de classificação

### Precisão, Recall, F1-Score e Suporte

In [88]:
# predição dos rótulos do test set
y_pred = clf.predict(X_test_pca)

# variáveis da performance do classificador
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      caneta       0.01      0.29      0.03         7
     chinelo       0.69      0.80      0.74        50
        copo       0.58      0.62      0.60       137
      escova       0.57      0.47      0.51       150
       leite       0.84      0.68      0.75       124
       livro       0.80      0.75      0.77       125
      oculos       0.67      0.46      0.55       126
     palheta       0.77      0.64      0.70       124
     tesoura       0.79      0.61      0.69       164

    accuracy                           0.61      1007
   macro avg       0.64      0.59      0.59      1007
weighted avg       0.71      0.61      0.65      1007



### Matriz de confusão
Um classificador perfeito apresenta números diferentes de zero somente na diagonal principal, e zero no restante.

In [89]:
print(confusion_matrix(y_test, y_pred))

[[  2   1   0   1   0   2   0   1   0]
 [  5  40   1   2   0   0   1   0   1]
 [ 16   2  85  14   1   6   7   3   3]
 [ 33   1  17  70   9   4   2   6   8]
 [ 14   0   4  12  84   8   0   1   1]
 [ 14   1   5   5   4  94   2   0   0]
 [ 29   5  19   6   1   1  58   3   4]
 [  7   3   8   6   1   2   8  79  10]
 [ 26   5   8   7   0   1   8   9 100]]
